In [1]:
import numpy as np 
import pandas as pd
from glob import glob
import json

In [2]:
class_df = pd.read_parquet('category_attributes.parquet', engine='pyarrow')
class_df

,Category,No_of_attribute,Attribute_list
0,Men Tshirts,5,"[color, neck, pattern, print_or_pattern_type, ..."
1,Sarees,10,"[blouse_pattern, border, border_width, color, ..."
2,Kurtis,9,"[color, fit_shape, length, occasion, ornamenta..."
3,Women Tshirts,8,"[color, fit_shape, length, pattern, print_or_p..."
4,Women Tops & Tunics,10,"[color, fit_shape, length, neck_collar, ocassi..."


In [3]:
class_dic = {}
for _,row_data in class_df.iterrows():
    class_dic[row_data['Category']]={}
    for idx,i in enumerate(list(row_data['Attribute_list'])):
        class_dic[row_data['Category']]['attr_'+str(idx+1)] = i
class_dic

{'Men Tshirts': {'attr_1': 'color',
  'attr_2': 'neck',
  'attr_3': 'pattern',
  'attr_4': 'print_or_pattern_type',
  'attr_5': 'sleeve_length'},
 'Sarees': {'attr_1': 'blouse_pattern',
  'attr_2': 'border',
  'attr_3': 'border_width',
  'attr_4': 'color',
  'attr_5': 'occasion',
  'attr_6': 'ornamentation',
  'attr_7': 'pallu_details',
  'attr_8': 'pattern',
  'attr_9': 'print_or_pattern_type',
  'attr_10': 'transparency'},
 'Kurtis': {'attr_1': 'color',
  'attr_2': 'fit_shape',
  'attr_3': 'length',
  'attr_4': 'occasion',
  'attr_5': 'ornamentation',
  'attr_6': 'pattern',
  'attr_7': 'print_or_pattern_type',
  'attr_8': 'sleeve_length',
  'attr_9': 'sleeve_styling'},
 'Women Tshirts': {'attr_1': 'color',
  'attr_2': 'fit_shape',
  'attr_3': 'length',
  'attr_4': 'pattern',
  'attr_5': 'print_or_pattern_type',
  'attr_6': 'sleeve_length',
  'attr_7': 'sleeve_styling',
  'attr_8': 'surface_styling'},
 'Women Tops & Tunics': {'attr_1': 'color',
  'attr_2': 'fit_shape',
  'attr_3': 'le

In [4]:
with open('prompts.json') as f:
    cat_dic = json.load(f)
cat_dic

{'Men Tshirts': {'color': {'default': 'A photo of a default colored Men T-shirt.',
   'multicolor': 'A photo of a multicolor Men T-shirt.',
   'black': 'A photo of a black colored Men T-shirt.',
   'white': 'A photo of a white colored Men T-shirt.'},
  'neck': {'round': 'A photo of a round neck Men T-shirt.',
   'polo': 'A photo of a polo neck Men T-shirt.'},
  'pattern': {'printed': 'A photo of a printed pattern Men T-shirt.',
   'solid': 'A photo of a solid pattern Men T-shirt.'},
  'print_or_pattern_type': {'default': 'A photo of a default print or pattern type Men T-shirt.',
   'solid': 'A photo of a solid print or pattern type Men T-shirt.',
   'typography': 'A photo of a typography print or pattern type Men T-shirt.'},
  'sleeve_length': {'short sleeves': 'A photo of a Men T-shirt with short sleeves.',
   'long sleeves': 'A photo of a Men T-shirt with long sleeves.'}},
 'Sarees': {'blouse_pattern': {'same as saree': 'A saree with a blouse pattern same as the saree.',
   'solid': 

In [9]:
train_df = pd.read_csv('train.csv').fillna('NA')
all_img_paths = glob('./train_images/*.jpg')
img_dic = {int(i.split('/')[-1].split('.jpg')[0]):i for i in all_img_paths}
# img_dic

In [6]:
train_df

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,NA,NA,NA,NA,NA
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NA,NA,NA,NA,NA
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NA,NA,NA,NA,NA
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NA,NA,NA,NA,NA
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70208,70374,Women Tops & Tunics,10,multicolor,fitted,regular,square neck,casual,printed,default,short sleeves,regular sleeves,ruffles
70209,70375,Women Tops & Tunics,10,yellow,regular,crop,round neck,casual,default,default,short sleeves,regular sleeves,knitted
70210,70376,Women Tops & Tunics,10,maroon,fitted,crop,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted
70211,70377,Women Tops & Tunics,10,NA,NA,NA,high,NA,NA,NA,short sleeves,NA,NA


In [7]:
dataset_list = []

In [10]:
for _,row_data in train_df.iterrows():
    attr_dic = cat_dic[row_data['Category']]
    img_path = img_dic[int(row_data['id'])]
    attr_set_dic = class_dic[row_data['Category']]
    for attr_id in attr_set_dic:
        temp_dic = {'img_path':'','caption':''}
        if row_data[attr_id]!='NA':
            caption = attr_dic[attr_set_dic[attr_id].replace('ocassion','occasion')][row_data[attr_id]]
            temp_dic['img_path'] = img_path
            temp_dic['caption'] = caption
            dataset_list.append(temp_dic)


In [51]:
# dataset_list

In [11]:
with open('cods_dataset_CLIP.json', "w") as f:
    written_count = 0
    for temp_dic in dataset_list:
        line_dict = {"image":temp_dic['img_path'], "caption":temp_dic['caption']}
        json_line = json.dumps(line_dict, indent=None, separators=(",",":"))
        #print(json_line)
        f.write(json_line + "\n")
        written_count += 1
